In [1]:
#reset variables
%reset -f

### Utility functions and Initial State

In [3]:
import csv
import matplotlib.image as mpimg
import cv2
import numpy as np
import pickle
import numpy as np
import math
from sklearn.utils import shuffle
# Fix error with TF and Keras
import tensorflow as tf

tf.python.control_flow_ops = tf

validation_data_folder_name = 'validation/'
validation_data_file_name = validation_data_folder_name + 'driving_log.csv'

data_folder_name = "data1-smooth/"
data_file_name = data_folder_name + 'driving_log.csv'

nrows = 40
ncols = 80

def get_image_path(img_name, is_validation_data = False):
    if is_validation_data:
        return validation_data_folder_name + img_name.strip()
    else:
        return data_folder_name + img_name.strip()

def read_image(img_name, is_validation_data = False):
    img_path = get_image_path(img_name, is_validation_data)
    img = mpimg.imread(img_path)
    img = cv2.resize(img, (ncols, nrows))
    return img

def count_imgs(is_validation_data = False):
    if is_validation_data:
        fname = validation_data_file_name
    else:
        fname = data_file_name
        
    count = 0
    
    with open(fname) as csv_file: 
        csv_reader = csv.reader(csv_file)

        for center, left, right, steering, throttle, brake, speed in csv_reader:
            if(center == 'center'):
                continue
                
            count += 1
            
    return count
    
def generate_batched_data_from_file(fname, batch_size=128, is_validation_data = False):
    
    while 1:
        csv_file = open(fname)
        csv_reader = csv.reader(csv_file)
        
        samples_count = 0;
        X = []
        y = []
        
        for center, left, right, steering, throttle, brake, speed in csv_reader:
            if(center == 'center'):
                continue
                
            X.append(read_image(center, is_validation_data))
            y.append(float(steering))
            
            samples_count += 1
            
            if samples_count >= batch_size:
                X, y = np.array(X), np.array(y)
                X, y = shuffle(X, y)
                yield(X, y)
                samples_count = 0
                X = []
                y = []

                
        csv_file.close()
        if samples_count > 0:
            X, y = np.array(X), np.array(y)
            X, y = shuffle(X, y)
            yield(X, y)
            
            
#read images count 
imgs_count = count_imgs()
validation_imgs_count = count_imgs(True)
print('imgs_count: ', imgs_count)
print('validation imgs count: ', validation_imgs_count)

imgs_count:  23777
validation imgs count:  2710


## Model Architecture Based on NVIDIA Pipeline 

In [3]:
from keras.models import Sequential
from keras.layers.core import Dense, Flatten, Activation, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.layers import Input

def train_model():
    n_model = Sequential()
    
    #Normalization layer with input shape = [?, 40, 80, 3]
    n_model.add(BatchNormalization(epsilon=0.001, mode=2, axis=1, input_shape=(nrows, ncols, 3)))
    
    #convolution layer with 'relu' activation and 24, [5, 5] kernels, [2, 2] stride, padding=same
    n_model.add(Convolution2D(24, 5, 5, subsample=(2,2), border_mode='same'))
    n_model.add(Activation('relu'))

    #convolution layer with 'relu' activation and 36, [5, 5] kernels, [2, 2] stride, padding=same
    n_model.add(Convolution2D(36, 5, 5, subsample=(2,2), border_mode='same'))
    n_model.add(Activation('relu'))
    #dropout to avoid overfitting with 0.75 prob
    n_model.add(Dropout(0.75))

    #convolution layer with 'relu' activation and 48, [5, 5] kernels, [2, 2] stride, padding=same
    n_model.add(Convolution2D(48, 5, 5, subsample=(2,2), border_mode='same'))
    n_model.add(Activation('relu'))

    #convolution layer with 'relu' activation and 64, [3, 3] kernels, [1, 1] stride, padding=valid
    n_model.add(Convolution2D(64, 3, 3, border_mode='valid', subsample=(1,1)))
    n_model.add(Activation('relu'))

    #convolution layer with 'relu' activation and 64, [3, 3] kernels, [1, 1] stride, padding=valid
    n_model.add(Convolution2D(64, 3, 3, border_mode='valid', subsample=(1,1)))
    n_model.add(Activation('relu'))

    #flatten layer
    n_model.add(Flatten())

    #fully connected layer with 'relu' activation and with output neurons count=1164
    n_model.add(Dense(1164))
    n_model.add(Activation('relu'))

    #fully connected layer with 'relu' activation and with output neurons count=100
    n_model.add(Dense(100))
    n_model.add(Activation('relu'))

    #fully connected layer with 'relu' activation and with output neurons count=50
    n_model.add(Dense(50))
    n_model.add(Activation('relu'))

    #fully connected layer with 'relu' activation and with output neurons count=10
    n_model.add(Dense(10))
    n_model.add(Activation('relu'))

    #fully connected layer with output neurons count=1
    n_model.add(Dense(1))
    
    #using 'Adam' optimizer with default hyperparams and using mean squared error as loss function
    n_model.compile(optimizer='adam', loss='mse', metrics=['mse', 'accuracy'])
    #this is just to avoid issues in case jupyter has cached any state.
    n_model.reset_states()
    
    #fit data using fit_generator and batch size = 128
    n_model.fit_generator(generate_batched_data_from_file(data_file_name), samples_per_epoch=imgs_count, nb_epoch=5,
                         validation_data=generate_batched_data_from_file(validation_data_file_name, is_validation_data=True),
                         nb_val_samples=validation_imgs_count)
    
    #save model architecture and weights for later use
    n_json_string = n_model.to_json()
    with open("model/model.json", 'w') as f:
        f.write(n_json_string)
    n_model.save_weights('model/model.h5')
    print('Model and weights saved!')

Using TensorFlow backend.


In [ ]:
train_model()

Epoch 1/10
38715/38715 [==============================] - 94s - loss: 0.0399 - mean_squared_error: 0.0399 - acc: 0.6426 - val_loss: 0.0168 - val_mean_squared_error: 0.0168 - val_acc: 0.6812

# Fine Tuning

At this point model has started working to some extent but car still goes out of the road at some points. To speed up tunning process I am going to fine tune already trained model above instead of training it from scratch. 

In [2]:
from keras.models import Sequential
from keras.layers.core import Dense, Flatten, Activation, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.models import model_from_json

def fine_tune_model(fname, samples_per_epoch, epochs=5):
    model_file_name = 'model/model.json'
    weights_file_name = 'model/model.h5'

    jfile = open(model_file_name, 'r')
    #load model architecture
    model = model_from_json(jfile.read())
    #compile model
    model.compile('adam', 'mse', ['mse', 'accuracy'])
    #load pre-trained weights so that we can fine tune them on new data
    model.load_weights(weights_file_name)
    #fine tune model based on new received data
    model.fit_generator(generate_batched_data_from_file(fname), samples_per_epoch=samples_per_epoch, nb_epoch=epochs,
                        validation_data=generate_batched_data_from_file(
                           validation_data_file_name, 
                           is_validation_data=True),
                        nb_val_samples=validation_imgs_count)

    #save model
    json_string = model.to_json()
    with open("model/model.json", 'w') as f:
        f.write(json_string)
    model.save_weights('model/model.h5')
    print('Model and weights saved!')

Using TensorFlow backend.


### Fine Tunning-1

In [ ]:
data_folder_name = "data2-provided/"
data_file_name = data_folder_name + 'driving_log.csv'

def fine_tune1():
    imgs_count = count_imgs()
    print('Imgs counted: ', imgs_count)
    fine_tune_model(data_file_name, imgs_count, epochs=5)
    
#call function 
fine_tune1()

### Fine Tunning-2

In [ ]:
data_folder_name = "data3-smooth/"
data_file_name = data_folder_name + 'driving_log.csv'
    
def fine_tune2():
    imgs_count = count_imgs()
    print('Imgs counted: ', imgs_count)
    fine_tune_model(data_file_name, imgs_count, epochs=5)

#call function
fine_tune2()

### Fine Tunning-3

Car having a lot of issues while passing through bridge. Sometimes it hits the bridge side and stays stuck there. Following fine tunning is specially to handle bridge case.

In [ ]:
data_folder_name = "data4-bridge/"
data_file_name = data_folder_name + 'driving_log.csv'

def fine_tune3():
    imgs_count = count_imgs()
    print('Imgs counted: ', imgs_count)
    fine_tune_model(data_file_name, imgs_count, epochs=5)
    fine_tune_model(data_file_name, imgs_count, epochs=5)

#call function
fine_tune3()

### Fine Tunning-4

Car still goes off the road at some edges it does not recognize so I am doing fine tunning of model for samples where car goes off the road.

In [ ]:
data_folder_name = "data5-edges/"
data_file_name = data_folder_name + 'driving_log.csv'

def fine_tune4():
    imgs_count = count_imgs()
    print('Imgs counted: ', imgs_count)
    fine_tune_model(data_file_name, imgs_count, epochs=5)
    
#call function
fine_tune4()

## Track-2 Fine-Tunning

Note: This fine-tunning was done after original model and code was submitted to Udacity and accepted. I also updated drive.py to increase speed as more speed was needed to cover steepest mountain paths. You can find old model in folder 'original-model-for-track1'

In [10]:
data_folder_name = "track-2/"
data_file_name = data_folder_name + 'driving_log.csv'

def fine_tune5():
    imgs_count = count_imgs()
    print('Imgs counted: ', imgs_count)
    fine_tune_model(data_file_name, imgs_count, epochs=5)
    
#call function
fine_tune5()
fine_tune5()
fine_tune5()
fine_tune5()
fine_tune5()
fine_tune5()
fine_tune5()
fine_tune5()

Imgs counted:  580
Epoch 1/5
580/580 [==============================] - 5s - loss: 0.0427 - mean_squared_error: 0.0427 - acc: 0.6034 - val_loss: 0.0520 - val_mean_squared_error: 0.0520 - val_acc: 0.6554
Epoch 2/5
580/580 [==============================] - 4s - loss: 0.0434 - mean_squared_error: 0.0434 - acc: 0.6000 - val_loss: 0.0435 - val_mean_squared_error: 0.0435 - val_acc: 0.6642

## Save Full Model

Saving full compiled model 

In [ ]:
def save_full_model():
    model_file_name = 'model/model.json'
    weights_file_name = 'model/model.h5'

    jfile = open(model_file_name, 'r')
    #load model architecture
    model = model_from_json(jfile.read())
    #compile model
    model.compile('adam', 'mse', ['mse', 'accuracy'])
    #load pre-trained weights so that we can fine tune them on new data
    model.load_weights(weights_file_name)
    
    model.save('full-model/model.h5')
    
#call function
save_full_model()

## Testing

In [ ]:
from keras.models import load_model

data_folder_name = "testing/"
data_file_name = data_folder_name + 'driving_log.csv'

def test_model():
    imgs_count = count_imgs()
    print('Imgs counted: ', imgs_count)
    
    model = load_model('full-model/model.h5')
    print('[Loss, MSE, Accuracy] = ', model.evaluate_generator(generate_batched_data_from_file(data_file_name), val_samples=imgs_count))
    
#call function
test_model()

## Visualizating Model

In [ ]:
from keras.utils.visualize_util import plot
def save_model_visualization():
    model = load_model('full-model/model.h5')
    plot(model, to_file='model.png')
    

#call function
save_model_visualization()

# Transfer Learning

In [ ]:
# from keras.applications.inception_v3 import InceptionV3
# from keras.models import Sequential, Model
# from keras.layers import Dense, GlobalAveragePooling2D
# from keras.layers.core import Dense, Flatten, Activation, Dropout
# from keras.layers.convolutional import Convolution2D
# from keras.layers.pooling import MaxPooling2D
# from keras.layers.normalization import BatchNormalization
# from keras.optimizers import Adam, SGD
# from keras.layers import Input

# def train_tl():

#     input_tensor = Input(shape=(150, 150, 3))
#     base_model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)

#     x = GlobalAveragePooling2D()(base_model.output)
#     d1 = Dense(50, activation='relu')(x)
#     d2 = Dense(10, activation = 'relu')(d1)
#     d3 = Dense(1)(d2)

#     tl_model = Model(input=base_model.input, output=d3)

#     for layer in base_model.layers:
#         layer.trainable = False

#     tl_model.compile(optimizer=SGD(lr=0.0001), loss='mse', metrics=['mse', 'accuracy'])
#     tl_model.fit_generator(generate_batched_data_from_file(data_file_name), samples_per_epoch=imgs_count, nb_epoch=2)

#     #save model arch and weights
#     tl_json_string = tl_model.to_json()
#     with open("model/model.json", 'w') as f:
#         f.write(tl_json_string)
#     tl_model.save_weights('model/model.h5')
#     print('Model and weights saved!')

In [ ]:
# train_tl()